Creating another gold table

In [0]:
silver_df = spark.table("workspace.default.silver_telco_churn")


In [0]:
silver_df.count()
silver_df.printSchema()


root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [0]:
from pyspark.sql.functions import when, col

features_df = (
    silver_df
    .withColumn(
        "SeniorCitizen",
        when(col("SeniorCitizen") == "Yes", 1)
        .when(col("SeniorCitizen") == "No", 0)
        .otherwise(None)
    )
    .select(
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents",
        "tenure",
        "PhoneService",
        "MultipleLines",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies",
        "Contract",
        "PaperlessBilling",
        "PaymentMethod",
        "MonthlyCharges",
        "TotalCharges",
        "Churn"
    )
)


In [0]:
features_df.write.mode("overwrite").saveAsTable(
    "workspace.default.gold_telco_churn_features"
)


STEP 4: Churn Prediction Model (Spark ML)
Predict whether a customer will churn (Yes/No) using historical data.

In [0]:
%sql
CREATE OR REPLACE TABLE gold_telco_churn_risk AS
SELECT
    customerID,
    tenure,
    Contract,
    InternetService,
    PaymentMethod,
    MonthlyCharges,
    TotalCharges,
    Churn,

    -- Risk score (business + AI logic)
    (
        CASE WHEN tenure < 12 THEN 2 ELSE 0 END +
        CASE WHEN Contract = 'Month-to-month' THEN 3 ELSE 0 END +
        CASE WHEN InternetService = 'Fiber optic' THEN 2 ELSE 0 END +
        CASE WHEN PaymentMethod LIKE '%electronic%' THEN 1 ELSE 0 END +
        CASE WHEN MonthlyCharges > 70 THEN 2 ELSE 0 END
    ) AS churn_risk_score

FROM silver_telco_churn;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE VIEW gold_telco_churn_decisions AS
SELECT
    *,
    CASE
        WHEN churn_risk_score >= 6 THEN 'HIGH_RISK'
        WHEN churn_risk_score BETWEEN 3 AND 5 THEN 'MEDIUM_RISK'
        ELSE 'LOW_RISK'
    END AS churn_risk_category
FROM gold_telco_churn_risk;


In [0]:
%sql
SELECT
    churn_risk_category,
    COUNT(*) AS customers,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM gold_telco_churn_decisions
GROUP BY churn_risk_category;


churn_risk_category,customers,percentage
MEDIUM_RISK,2719,38.61
LOW_RISK,2193,31.14
HIGH_RISK,2131,30.26


In [0]:
%sql
SELECT
    churn_risk_category,
    AVG(MonthlyCharges) AS avg_monthly_charges,
    AVG(tenure) AS avg_tenure
FROM gold_telco_churn_decisions
GROUP BY churn_risk_category;


churn_risk_category,avg_monthly_charges,avg_tenure
MEDIUM_RISK,62.64192717910966,29.62449429937477
LOW_RISK,45.68561331509347,46.54354765161879
HIGH_RISK,87.09744251525102,21.29094321914594
